# Traffic Application Testing Code

This notebook is designed to communicate with the Traffic Application server and perform a series of tests. You should start the server with a copy of the clean database and execute the core code and each of the tests. If a test fails, it is necessary to restart the server and re-run the test to ensure that the test did not fail because of the state the server was in due to the previous tests.

In [7]:
# The core code.
import random
import requests
import xml.etree.ElementTree as ET
import base64

def parse_actions(root):
    actions = []
    for child in root.findall('action'):
        type = child.find('type')
        if(type.text == 'refill'):
            where = child.find('where')
            what = child.find('what')
            actions.append(['refill',[where.text,base64.b64decode(what.text).decode('ascii')]])
        if(type.text == 'redirect'):
            where = child.find('where')
            actions.append(['redirect',[where.text]])
        if(type.text == 'append'):
            where = child.find('where')
            what = child.find('what')
            actions.append(['refill',[where.text,base64.b64decode(what.text).decode('ascii')]])
    return actions

def request2server(url,cookies):
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]))
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        root=ET.fromstring(r.text)
        act = parse_actions(root)
        return [[ucookie,mcookie],act]
    except:
        return [[ucookie,mcookie],[['status', [-1]]]]
    
def find_redirect(act):
    for a in act:
        if a[0] == 'redirect':
            return a[1][0]
    return None

def find_refill(act,where):
    for a in act:
        if a[0] == 'refill':
            if a[1][0] == where:
                return a[1][1]
    return None

def randparam():
    return "&rand="+str(random.randrange(10000,99999))

def do_login(cookies,user,pasw):
    return request2server("http://localhost:8081/action?command=login&usernameinput="+user+"&passwordinput="+pasw+randparam(),cookies)

def do_add(cookies,loc,occupancy,vtype):
    return request2server("http://localhost:8081/action?command=add&locationinput="+loc+"&occupancyinput="+str(occupancy)+"&typeinput="+vtype+randparam(),cookies)

def do_summary(cookies):
    return request2server("http://localhost:8081/action?command=summary"+randparam(),cookies)

def do_logout(cookies):
    return request2server("http://localhost:8081/action?command=logout"+randparam(),cookies)

In [8]:
# Test 1 - Simple login
def test1():
    try:
        cookies = ['','']
        [cookies,act] = do_login(cookies,"test1","password1")
    
        where = find_redirect(act)
    
        if (where != '/page.html'):
            print("Test 1 Failed - Expected /page.html got {"+where+"}")
            return

        [cookies,act] = do_logout(cookies)
        
        where = find_redirect(act)
    
        if (where != '/index.html'):
            print("Test 1 Failed - Expected /index.html got {"+where+"}")
            return
        print("Test 1 Passed")
    except:
        print("Test 1 Failed - Exception Caused.")
        

In [9]:
# Test 2 - Bad login
def test2():
    try:
        cookies = ['','']
        [cookies,act2] = do_login(cookies,"test1","passwor1")
        where = find_redirect(act2)
    
        if (where != None):
            print("Test 2 Failed - Unexpected redirect.")
            return
        
        message = find_refill(act2,'message')
    
        if (message == None):
            print("Test 2 Failed - No failure message present")
            return
        
        [cookies,act2] = do_logout(cookies)
        
        where = find_redirect(act2)
    
        if (where != '/index.html'):
            print("Test 2 Failed - Expected /index.html got {"+where+"}")
            return
        
        print("Test 2 Passed")
    except:
        print("Test 2 Failed - Exception Caused.")
        

In [10]:
# Test 3 - Injection attack
def test3():
    try:
        cookies = ['','']
        [cookies,act2] = do_login(cookies,"test1",'"'+' OR "0"="0')
        where = find_redirect(act2)
    
        if (where != None):
            print("Test 3 Failed - Unexpected redirect.")
            return
        
        message = find_refill(act2,'message')
    
        if (message == None):
            print("Test 3 Failed - No failure message present")
            return
        
        [cookies,act2] = do_logout(cookies)
        
        where = find_redirect(act2)
    
        if (where != '/index.html'):
            print("Test 3 Failed - Expected /index.html got {"+where+"}")
            return
        
        print("Test 3 Passed")
    except:
        print("Test 3 Failed - Exception Caused.")

In [11]:
# Test 4 - Simple add
def test4():
    try:
        cookies = ['','']
        [cookies,act] = do_login(cookies,"test1","password1")
        where = find_redirect(act)
    
        if (where != '/page.html'):
            print("Test 4 Failed - Unexpected redirect.(1)")
            return
        
        [cookies,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 4 Failed - Unexpected redirect.(2)")
            return
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 4 Failed - No acknowledge message present")
            return
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 4 Failed - No total message present")
            return
        
        if (total != "1"):
            print("Test 4 Failed - Total not expected value")
            return
        
        [cookies,act] = do_logout(cookies)
        
        where = find_redirect(act)
    
        if (where != '/index.html'):
            print("Test 4 Failed - Expected /index.html got {"+where+"}")
            return
        
        print("Test 4 Passed")
    except:
        print("Test 4 Failed - Exception Caused.")

In [12]:
# Run all tests
test1()
test2()
test3()
test4()

Test 1 Failed - Exception Caused.
Test 2 Failed - No failure message present
Test 3 Failed - No failure message present
Test 4 Failed - Unexpected redirect.(1)
